<a href="https://colab.research.google.com/github/GiovanniBorrelli/RetiGeografiche/blob/main/ParoleNapoletanePresenti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import openpyxl
import string
from tqdm import tqdm
import unicodedata

def clean_text(text):
    # Rimuovi la punteggiatura, le virgolette e il carattere speciale "…" (ignora l'apostrofo) e converti in minuscolo
    translator = str.maketrans('', '', string.punctuation.replace("'", "") + '“”…')
    cleaned_text = text.translate(translator).lower()

    # Rimuovi gli accenti
    cleaned_text = ''.join(char for char in unicodedata.normalize('NFD', cleaned_text) if unicodedata.category(char) != 'Mn')

    # Tratta i numeri come parole presenti nel dizionario
    cleaned_text = ' '.join(word if word.isalpha() else 'numero' for word in cleaned_text.split())

    return cleaned_text

def count_words_in_sentence(sentence, dictionary_words):
    # Conta il numero di parole presenti e non presenti nel dizionario
    words = clean_text(sentence).split()
    word_count = {}
    for word in words:
        word_count[word] = word_count.get(word, 0) + 1

    present_words = sum(word_count[word] for word in word_count if word in dictionary_words)
    absent_words = sum(word_count[word] for word in word_count if word not in dictionary_words)

    return present_words, absent_words

def main(dictionary_file, excel_file, output_file, column_name):
    # Carica il dizionario e crea un set di parole senza accenti
    dictionary_words = count_words_in_dictionary(dictionary_file)

    # Apre il file Excel e ottiene il foglio di lavoro
    workbook = openpyxl.load_workbook(excel_file)
    sheet = workbook.active

    # Trova l'indice della colonna
    column_index = None
    for col in range(1, sheet.max_column + 1):
        if sheet.cell(row=1, column=col).value == column_name:
            column_index = col
            break

    if column_index is None:
        print(f"Colonna '{column_name}' non trovata.")
        return

    # Aggiungi intestazioni per il conteggio delle parole
    sheet.cell(row=1, column=column_index + 1).value = "Parole nel dizionario"
    sheet.cell(row=1, column=column_index + 2).value = "Parole assenti nel dizionario"

    # Conta le parole per ogni frase
    for row in tqdm(range(2, sheet.max_row + 1), desc="Elaborazione"):
        sentence = sheet.cell(row=row, column=column_index).value

        # Verifica se la frase è presente
        if sentence is not None:
            present_words, absent_words = count_words_in_sentence(sentence, dictionary_words)
            sheet.cell(row=row, column=column_index + 1).value = present_words
            sheet.cell(row=row, column=column_index + 2).value = absent_words

    # Salva le modifiche
    workbook.save(output_file)

def count_words_in_dictionary(dictionary_file):
    # Leggi il dizionario e restituisci un set di parole senza accenti
    with open(dictionary_file, 'r', encoding='utf-8') as file:
        dictionary_words = {clean_text(word.strip()) for word in file}
    return dictionary_words

if __name__ == "__main__":
    dictionary_file = "dictionary.txt"
    excel_file = "RisultatoCompleto.xlsx"
    output_file = "risultato_valutazione9.xlsx"
    column_name = "Frasi"  # Specifica il nome della colonna da analizzare
    main(dictionary_file, excel_file, output_file, column_name)


Elaborazione: 100%|██████████| 29828/29828 [00:01<00:00, 15902.27it/s]
